In [2]:
import random

# サイコロの問題
def rollDie():
    return random.choice([1,2,3,4,5,6])

def checkPascal(numTrials):
    numWins = 0
    for i in range(numTrials):
        for j in range(24):
            d1 = rollDie()
            d2 = rollDie()
            if d1 == 6 and d2 == 6:
                numWins += 1
                break
    print('Probability of winning = ', numWins/numTrials)
    
checkPascal(10000)

Probability of winning =  0.4819


In [76]:
import pandas as pd
import pprint
import numpy as np
import math

def variance(X):
    mean = sum(X)/len(X)
    tot = 0.0
    for x in X:
        tot += (x - mean)**2
    return tot/len(X)

def stdDev(X):
    return math.sqrt(variance(X))

# クラップスと呼ばれるゲーム
class CrapsGame(object):
    def __init__(self):
        self.passWins, self.passLosses = 0, 0
        self.dpWins, self.dpLosses, self.dpPushes = 0, 0, 0
    
    def playHand(self):
        throw = rollDie() + rollDie()
        if throw == 7 or throw == 11:
            self.passWins += 1
            self.dpLosses += 1
        elif throw == 2 or throw == 3 or throw == 12:
            self.passLosses += 1
            if throw == 12:
                self.dpPushes += 1
            else:
                self.dpWins += 1
        else:
            point = throw
            while True: # pointごとの勝率を事前計算しておけば、乱数と比較する処理で内部ループを置き換えることができる
                throw = rollDie() + rollDie()
                if throw == point:
                    self.passWins += 1
                    self.dpLosses += 1
                    break
                elif throw == 7:
                    self.passLosses += 1
                    self.dpWins += 1
                    break
    
    def passResults(self):
        return (self.passWins, self.passLosses)
    
    def dpResults(self):
        return (self.dpWins, self.dpLosses, self.dpPushes)

def crapsSim(handsPerGame, numGames):
    games = []
    for t in range(numGames):
        c = CrapsGame()
        for i in range(handsPerGame):
            c.playHand()
        games.append(c)
    
    pROIPerGame, dpROIPerGame = [], []
    for g in games:
        wins, losses = g.passResults()
        pROIPerGame.append((wins - losses)/float(handsPerGame))
        wins, losses, pushes = g.dpResults()
        dpROIPerGame.append((wins - losses)/float(handsPerGame))
    meanROI = str(round(100*sum(pROIPerGame)/numGames, 4)) + '%'
    sigma = str(round(100*stdDev(pROIPerGame), 4)) + '%'
    print('Pass:', 'Mean ROI = ', meanROI, 'Std. Dev. = ', sigma)
    meanROI = str(round((100*sum(dpROIPerGame)/numGames), 4)) + '%'
    sigma = str(round(100*stdDev(dpROIPerGame), 4)) + '%'
    print('Don\'t pass:', 'Mean ROI =', meanROI, 'Std Dev = ', sigma)
    
    print('---')
    print(pd.DataFrame(pROIPerGame).std()[0])
    print(pd.DataFrame(dpROIPerGame).std()[0])
    
    # 標本分散
    print('---')
    print(pd.DataFrame(pROIPerGame).std(ddof=0)[0])
    print(pd.DataFrame(dpROIPerGame).std(ddof=0)[0])
    
    print('---')
    print(np.std(pROIPerGame))
    print(np.std(dpROIPerGame))
    
    # 不偏分散
    print('---')
    print(np.std(pROIPerGame, ddof=1))
    print(np.std(dpROIPerGame, ddof=1))
    
    # 手計算
    def stdDeviance(data, bbof=0):
        denom = len(pROIPerGame)
        m = np.mean(pROIPerGame)
        if bbof == 1:
            denom -= 1
        return (pd.DataFrame(pROIPerGame).apply(lambda x: (m - x)**2).sum()[0]/denom)**0.5
    
    print('---')
    std = stdDeviance(pROIPerGame)
    std2 = stdDeviance(pROIPerGame, bbof=1)
    print("Pass: Std =", std, "Unbiased Std =", std2)
    std = stdDeviance(dpROIPerGame)
    std2 = stdDeviance(dpROIPerGame, bbof=1)
    print("Don't Pass: Std =", std, "Unbiased Std =", std2)

crapsSim(100, 100)

Pass: Mean ROI =  -0.84% Std. Dev. =  9.3217%
Don't pass: Mean ROI = -1.93% Std Dev =  9.038%
---
0.0936867766614
0.0908351264766
---
0.0932171658012
0.0903798096922
---
0.0932171658012
0.0903798096922
---
0.0936867766614
0.0908351264766
---
Pass: Std = 0.0932171658012 Unbiased Std = 0.0936867766614
Don't Pass: Std = 0.0932171658012 Unbiased Std = 0.0936867766614
